In [ ]:
"""
Purpose: To pull down the latest segment ids that 
correspond to the nucleus table so they can be used to 
run downstream tables

"""

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from os import sys
sys.path.append("/meshAfterParty/")

In [6]:
import datajoint_utils as du
nucleus_table = du.configure_nucleus_table()
nucleus_table

INFO - 2021-01-14 19:41:55,092 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-14 19:41:55,093 - settings - Setting database.user to celiib
INFO - 2021-01-14 19:41:55,093 - settings - Setting database.password to newceliipass
INFO - 2021-01-14 19:41:55,097 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-14 19:41:55,097 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-14 19:41:55,339 - settings - Setting e

ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.,"nucleus_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",supervoxel_id id of the supervoxel under the nucleus centroid. Equivalent to Allen: 'pt_supervoxel_id'.,volume volume of the nucleus in um^3
30.00,996,0,60464,93616,20968,0,35.1478
30.00,1833,0,56800,97280,19929,0,35.934
30.00,1841,0,57536,105584,19883,0,265.586
30.00,1896,0,59872,96608,19853,0,174.558
30.00,1998,864691136050815731,59936,105872,20078,72978435697419638,137.669
30.00,2005,0,61472,96128,20003,0,71.6715
30.00,2020,0,62448,97920,20177,0,214.823
30.00,2026,0,60832,100752,19857,0,103.014
30.00,2051,0,61968,96704,20133,0,38.5804
30.00,2198,864691135084884330,65088,104160,20059,73752285724957558,76.0236


In [7]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-14 19:41:55,580 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-14 19:41:55,842 - settings - Setting enable_python_native_blobs to True


In [ ]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30

key_source =  ((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                        "decimation_version=" + str(decimation_version) &
                   f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") & (minnie.OldBaylorSegmentCentroid() & "multiplicity<3").proj()
                                                             & (dj.U("segment_id") & nucleus_table))
key_source

In [ ]:
new_seg_ids = (dj.U("segment_id") & nucleus_table).fetch("segment_id")

In [ ]:
filtered_seg_ids = new_seg_ids[new_seg_ids!= 0]

In [ ]:
filtered_seg_ids_dict = [dict(segment_id=k) for k in filtered_seg_ids]

In [ ]:
@schema
class NucleiSegmentsRun2(dj.Manual):
    definition="""
    segment_id : bigint unsigned  #segment id for those to be decimated
    """
    
NucleiSegmentsRun2.insert(filtered_seg_ids_dict,skip_duplicates=True)

# Figuring Out What Meshes we don't have

In [3]:
import datajoint as dj
version = 30
m65mat = dj.create_virtual_module('m65mat', 'microns_minnie65_materialization')
nucleus_table_ids = m65mat.Nucleus.Info & {'ver': version} & "segment_id>0"

ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.,"nucleus_x x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_y y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)","nucleus_z z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)",supervoxel_id id of the supervoxel under the nucleus centroid. Equivalent to Allen: 'pt_supervoxel_id'.,volume volume of the nucleus in um^3
30.00,1998,864691136050815731,59936,105872,20078,72978435697419638,137.669
30.00,2198,864691135084884330,65088,104160,20059,73752285724957558,76.0236
30.00,2199,864691136521572241,65664,109488,19839,73823341596728836,142.404
30.00,2389,864691135494586958,58256,103152,20676,72766985934597733,50.0802
30.00,2398,864691136451074559,58720,109408,20589,72838179312529142,131.664
30.00,2433,864691135524436073,59968,109952,21002,73049354331530450,126.039
30.00,3024,864691135087399350,57024,103536,21200,72626248513279322,238.007
30.00,3117,864691134684964653,58512,105424,21317,72837629623886011,244.205
30.00,3121,864691132625568958,57648,103360,21760,72696617324576355,30.9133
30.00,3152,864691135524409705,58736,107008,21323,72837835782285934,47.1471


In [10]:
changed_ids = nucleus_table_ids.proj() - minnie.Segment

In [12]:
need_to_download_mesh_ids = changed_ids.fetch("segment_id")
need_to_download_mesh_ids

array([864691132625568958, 864691136119039313, 864691136723556861, ...,
       864691134851052247, 864691134344325456, 864691136069916432])

In [13]:
import numpy as np
np.savez("Mesh_ids_to_Download.npz",segment_ids=need_to_download_mesh_ids)

In [15]:
np.load("Mesh_ids_to_Download.npz",allow_pickle=True)["segment_ids"]

array([864691132625568958, 864691136119039313, 864691136723556861, ...,
       864691134851052247, 864691134344325456, 864691136069916432])